In [4]:
import json
import nltk

s_tokenizer = nltk.RegexpTokenizer('[\w\' ]+')
w_tokenizer = nltk.RegexpTokenizer('[\w\']+')

In [5]:
def tokenize_to_word(content):
    content = content.replace('\n', '.').replace("\r", ".").lower()
    ss = s_tokenizer.tokenize(content)
    r = []
    for s in ss:
        r.append(w_tokenizer.tokenize(s))
    return r

In [6]:
def LCS(x, y):
    dp = [[0 for _ in y] for _ in x]
    for i in range(len(x)):
        for j in range(len(y)):
            if i > 0:
                dp[i][j] =  max(dp[i][j], dp[i - 1][j])
            if j > 0:
                dp[i][j] = max(dp[i][j], dp[i][j - 1])
            if x[i] == y[j]:
                if i > 0 and j > 0: dp[i][j] = max(dp[i][j], dp[i - 1][j - 1] + 1)
                else: dp[i][j] = 1   
    maxx = 0
    for i in range(len(x)):
        for j in range(len(y)):
            maxx = max(maxx, dp[i][j])
    return maxx

def rouge(rx, ry):
    #global lx, ly
    #lx += len(x)
    #ly += len(y)
    x = rx.lower().split()
    y = ry.lower().split()
    lcs = LCS(x, y)
    if lcs == 0: return 0
    recall = lcs * 1.0 / len(y)
    precision = lcs * 1.0 / len(x)
    beta = precision / (recall + 1e-12)
    return (1 + beta * beta) * recall * precision / (recall + beta * beta * precision)

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [8]:
def tokenize_to_word(content):
    return [[]]
    r = [[]]
    s = nlp(content.lower())
    for token in s.doc:
        if token.pos_ == 'PUNCT' and token.string != '-':
            if len(r[-1]) != 0:
                r.append([])
            continue
        if not token.is_stop:
            #r[-1].append((token.string, token.pos_))
            r[-1].append(token.string)
    return r

In [9]:
def load(fnames, test=False):
    for fname in fnames:
        with open(fname) as f:
            for line in f:
                obj = json.loads(line)
                if not test and len(obj['content']) > 2000:
                    continue
                yield tokenize_to_word(obj['content']), obj.get('title'), obj['id'], obj['content']

In [10]:
from summa.summarizer import summarize
from summa import keywords

In [19]:
data = load(["data/raw/bytecup.corpus.train.0.txt"])
cnt = 0
sa, sb = 0, 0
for sentences, title , _, raw in data: 
    if cnt > 1000: break
    cnt += 1
    doc = nlp(raw)
    r = []
    for ent in doc.ents[: 5]:
        r.append(ent.text)
    
    first_sentence = s_tokenizer.tokenize(raw)[0]
    front_keywords = " ".join(r)
    
    ta = rouge(title, first_sentence)
    tb = rouge(title, front_keywords)
    sa += ta
    sb += tb
    #print(title)
    #print(first_sentence)
    #print(front_keywords)
    #print(ta, tb)
    

In [20]:
sa, sb

(128.80138713973716, 154.63617330917182)

In [152]:
nlp = spacy.load('en_core_web_sm')
doc = nlp("There are a few topics that Bill Belichick will speak at length about ")
for ent in doc.ents:
    print(ent.text)

Bill Belichick


In [89]:
from collections import defaultdict
def textrank(sentences, w = 3):
    sentences_id = []
    words = {}
    for sentence in sentences:
        sentences_id.append([])
        for i, token in enumerate(sentence):
            if token not in words:
                words[token] = len(words)
            sentences_id[-1].append(words[token])
    
    edges_map = [defaultdict(float) for _ in words]
    for sid in sentences_id:
        for i in range(len(sid)):
            for j in range(-w, w):
                if i + j >= 0 and i + j < len(sid) and j != 0:
                    edges_map[sid[i]][sid[i + j]] += 1
    for edges in edges_map:
        total = 0
        for v in edges: total += edges[v]
        for v in edges: edges[v] /= total
    
    values = graph_solve(edges_map)
    words_value  = {}
    for word in words:
        words_value[word] = values[words[word]]
    
    r = []
    for sentence in sentences:
        value = 0
        for word in sentence: value += words_value[word]
        value /= len(sentence) + 5
        r.append((" ".join(sentence), value))
    r.sort(key=lambda o: o[1], reverse=True)
    return r
    
def graph_solve(edges_map, iter_num = 100, discount=0.85):
    n = len(edges_map)
    values = [100 for i in range(n)]
    for it in range(iter_num):
        t = [0 for i in range(n)]
        for u, edges in enumerate(edges_map):
            for v in edges:
                weight = edges[v]
                t[v] += (1 - discount) + discount * values[u] * weight;
        w = t
    return w

In [92]:
data = load(["data/raw/bytecup.corpus.train.0.txt"])
for sentences, title , _ in data:
    
    print(title)
    r = textrank(sentences)
    break

The Latest: Biden says on MLK day American values challenged


In [93]:
con

[("Maryland  pastor  denounced  President  Donald  Trump 's  vulgar  description  African  nations",
  101.08881448412697),
 ("The  News  Journal  reports  Biden  criticized  President  Donald  Trump 's  response  white  nationalist  rallies  August  Charlottesville  reported  use  obscenity  describe  African  nations  Haiti  recent  immigration  discussion  Congressional  leaders",
  99.00104432604431),
 ("Biden  keynote  speaker  Monday  Delaware  State  Bar  Association 's  annual  Martin  Luther  King  Jr.  Day  breakfast  Wilmington",
  94.15939262757446),
 ('Former  Vice  President  Joe  Biden  says  American  values  challenged  present  times  leadership  Martin  Luther  King  Jr.  provides  example  respond',
  89.44961242461244),
 ("But  said  believes  Americans  ready  respond  - establish  nation 's ",
  85.96223128723129),
 ('WUSA - TV  reports  Pence  red - faced  times  sermon', 84.90510637510639),
 ("Watson  called  Trump 's  remarks ", 78.72928571428571),
 ("Pence 's

In [1]:
import _thread

In [15]:
def calc(sentences, iid):
    abstracts = summarize(' . '.join(sentences), scores=True)
    sorted(abstracts, key=lambda o:o[1], reverse=True)
    with open('data/result/%s.txt' % iid, 'w') as fout:
        print(abstracts[0][0][:-1], file=fout)

def calc(raw, iid):
    doc = nlp(raw)
    r = []
    for ent in doc.ents[: 5]:
        r.append(ent.text)
    with open('data/result/%s.txt' % iid, 'w') as fout:
        print(" ".join(r).strip().lower(), file=fout)

In [16]:
test = load(["data/raw/bytecup.corpus.validation_set.txt"], test=True)
for sentences, title, iid, raw in test:
    calc(raw, iid)
    #calc(sentences, iid)
    #_thread.start_new_thread(calc, (raw, iid))